In [6]:
import torchvision.transforms as transforms
from dataset_trucks import ImageClassificationDataset

TASK = 'trucks'
# TASK = 'emotions'
# TASK = 'fingers'
# TASK = 'diy'

CATEGORIES = ['cemento', 'no_truck','otro','plataforma','volteo','volteo_corto']
# CATEGORIES = ['none', 'happy', 'sad', 'angry']
# CATEGORIES = ['1', '2', '3', '4', '5']
# CATEGORIES = [ 'diy_1', 'diy_2', 'diy_3']

DATASETS = ['A']
# DATASETS = ['A', 'B', 'C']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = ImageClassificationDataset('./data', CATEGORIES, TRANSFORMS)
    
print("{} task with {} categories defined".format(TASK, CATEGORIES))

trucks task with ['cemento', 'no_truck', 'otro', 'plataforma', 'volteo', 'volteo_corto'] categories defined


In [7]:
def save_model(path):
    torch.save(model.state_dict(), path)

In [8]:
import torch
import torchvision


device = torch.device('cuda')

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, len(dataset.categories))

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, len(dataset.categories), kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
# model = torchvision.models.resnet18(pretrained=True)
# model.fc = torch.nn.Linear(512, len(dataset.categories))
categories = 6
# RESNET 34
model = torchvision.models.resnet34(pretrained=True)
model.fc = torch.nn.Linear(512, categories)
    
model = model.to(device)

# display(model_widget)
print("model configured and model_widget created")

model configured and model_widget created


In [9]:
dataset = datasets[DATASETS[0]]


In [10]:
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())

In [19]:
import ipywidgets
import threading
import time
import torch.nn.functional as F
epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
accuracy_widget = ipywidgets.FloatText(description='accuracy')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer,epochs_widget, accuracy_widget, loss_widget, progress_widget
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()
        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, labels in iter(train_loader):
                # send data to device
                images = images.to(device)
                labels = labels.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute loss
                loss = F.cross_entropy(outputs, labels)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                error_count += len(torch.nonzero(outputs.argmax(1) - labels).flatten())
                count = len(labels.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                accuracy_widget.value = 1.0 - error_count / i
                
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except e:
        pass
    model = model.eval()

In [20]:
from IPython.display import display

In [21]:

train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    accuracy_widget,
    ipywidgets.HBox([train_button, eval_button])
])
# Combine all the widgets into one display
all_widget = ipywidgets.VBox([ 
    train_eval_widget,
])

display(all_widget)

In [34]:
train_eval(True)

In [35]:
save_model('truck_classifier_resnet34_torch_100.pth')

In [54]:
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision
import cv2
import PIL.Image
import numpy as np


class TruckClassifierTorch():
    def __init__(self,categories,model_path):
        self.device = torch.device('cuda')
        self.mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
        self.std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
        self.categories = categories
        
        self.model = torchvision.models.resnet34(pretrained=False)
        self.model.fc = torch.nn.Linear(512, self.categories)
        self.model = self.model.to(device)
        self.model.load_state_dict(torch.load(model_path))
        # ALEXNET
        # model = torchvision.models.alexnet(pretrained=True)
        # model.classifier[-1] = torch.nn.Linear(4096, len(dataset.categories))

        # SQUEEZENET 
        # model = torchvision.models.squeezenet1_1(pretrained=True)
        # model.classifier[1] = torch.nn.Conv2d(512, len(dataset.categories), kernel_size=1)
        # model.num_classes = len(dataset.categories)

        # RESNET 18
        # model = torchvision.models.resnet18(pretrained=True)
        # model.fc = torch.nn.Linear(512, len(dataset.categories))

    def preprocess(self,image):
        image = PIL.Image.fromarray(image)
        image = transforms.functional.to_tensor(image).to(self.device)
        image.sub_(self.mean[:, None, None]).div_(self.std[:, None, None])
        return image[None, ...]
        
    def detect(self,image):
        # Inference
        preprocessed = self.preprocess(image)
        output = self.model(preprocessed)
        output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
        return output

In [63]:
tc = TruckClassifierTorch(6,'truck_classifier_resnet34_torch.pth')
img = cv2.imread('exit_gate_20210619070712_5ab48baa-68fc-4a8e-9f69-6047ffca612f_TRUCK.jpg')

output = tc.detect(img)
print(output)
print(CATEGORIES[np.argmax(output)])

[0.12155704 0.00204893 0.05717224 0.00131879 0.81596553 0.00193751]
volteo


['cemento', 'no_truck', 'otro', 'plataforma', 'volteo', 'volteo_corto']

In [68]:
import os
for i in os.listdir('data'):
    print(i,len(os.listdir(f'data/{i}')))

volteo_corto 190
no_truck 323
otro 214
cemento 532
plataforma 53
volteo 995
